## Import some needed scripts

In [ ]:
from dataset import getFeaturesFromDBData, getDataFromTweeterInDB, Database
from pandas import DataFrame
from pprint import pprint

## Define some constants needed

In [ ]:
MONGO_DB_URI = "mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false"
USER_DATASET_FILE = "users_dataset.csv"

## Gather data using twitter API and populate MongoDB

We gather the data in a way to try to maintain a 50-50% of bots and humans

In [ ]:
getDataFromTweeterInDB(MONGO_DB_URI,USER_DATASET_FILE)

In [ ]:
dataset = getFeaturesFromDBData(MONGO_DB_URI)
dataset.to_csv("dataset.csv")
pprint(dataset.head())